# Indefinite Admittance Matrix

> Refs:
> * High Frequency Amplifiers by Ralph S. Carson [page 7...]

$$
\begin{bmatrix}
I_1 \\
I_2 \\
I_3
\end{bmatrix} = 
\begin{bmatrix}
y_{11} & y_{12} & y_{13} \\
y_{21} & y_{22} & y_{23} \\
y_{31} & y_{32} & y_{33} \\
\end{bmatrix}
\begin{bmatrix}
V_1 \\
V_2 \\
V_3
\end{bmatrix}
$$

The sum of any row or column adds up to zero

$$
I_1 + I_2 + I_3 = 0
$$

$$
y_{11} + y_{12} + y_{13} = 0
$$

$$
y_{11} + y_{21} + y_{31} = 0
$$

In [111]:
import math
import sys

import numpy as np
import pandas as pd
from IPython.display import HTML, display

sys.path.append("../lib")
from ycx_complex_numbers import NetY, Y


def tabulate_indefinite_admittance_matrix(
    n, labels=["Base", "Emitter", "Collector"], common_idx=1
):
    ndf = pd.DataFrame(
        [n.y11, n.y12, n.y21, n.y22],
        index=["y11", "y12", "y21", "y22"],
        columns=[f"Common-{labels[common_idx]}"],
    )
    display(
        HTML(
            f"<h3>Provided matrix for configuration 'Common-{labels[common_idx]}':-</h3>"
        )
    )
    display(ndf.style.format("{:.6f}"))

    n = n.m

    d = np.full(
        (3, 3),
        math.nan,
        dtype=Y,
    )
    nroff = 0
    ncoff = 0
    for r in [0, 1, 2]:
        if r == common_idx:
            pass
        else:
            for c in [0, 1, 2]:
                if c == common_idx:
                    pass
                else:
                    d[r][c] = n[nroff][ncoff]
                    ncoff += 1
            nroff += 1
        ncoff = 0

    ######################
    # calc missing values

    df = pd.DataFrame(d, columns=labels, index=labels)
    display(
        HTML(
            f"<h3>Provided matrix mapped to table as 'Common-{labels[common_idx]}':-</h3>"
        )
    )
    display(df.style.format("{:.6f}"))

    # sum nan entries per row
    r_nans = df.isna().sum(axis=0)
    for r in [0, 1, 2]:
        if r_nans.iloc[r] == 1:  # one missing value to resolve
            resolv_idx = -1
            sum_flds = Y(0 + 0j)
            for c in [0, 1, 2]:
                if isinstance(df.values[r][c], Y):
                    sum_flds += df.values[r][c]
                else:
                    resolv_idx = c
            if resolv_idx > -1:
                df.values[r][resolv_idx] = -sum_flds

    # sum nan entries per column
    c_nans = df.isna().sum(axis=0)
    for c in [0, 1, 2]:
        if c_nans.iloc[c] == 1:  # one missing value to resolve
            resolv_idx = -1
            sum_flds = Y(0 + 0j)
            for r in [0, 1, 2]:
                if isinstance(df.values[r][c], Y):
                    sum_flds += df.values[r][c]
                else:
                    resolv_idx = r
            if resolv_idx > -1:
                df.values[resolv_idx][c] = -sum_flds

    s = df.style.format("{:.6f}")
    display(HTML("<h3>Populated with missing values:-</h3>"))
    display(s)


tabulate_indefinite_admittance_matrix(
    NetY(
        y11=(2 + 2j) * 10**-3,
        y12=(-2 - 20j) * 10**-6,
        y21=(20 - 3j) * 10**-3,
        y22=(20 + 60j) * 10**-6,
    )
)

,Common-Emitter
y11,Y:0.002000+0.002000j : [mag:0.002828 ∠45.000000]
y12,Y:-0.000002-0.000020j : [mag:0.000020 ∠-95.710593]
y21,Y:0.020000-0.003000j : [mag:0.020224 ∠-8.530766]
y22,Y:0.000020+0.000060j : [mag:0.000063 ∠71.565051]


,Base,Emitter,Collector
Base,Y:0.002000+0.002000j : [mag:0.002828 ∠45.000000],nan,Y:-0.000002-0.000020j : [mag:0.000020 ∠-95.710593]
Emitter,nan,nan,nan
Collector,Y:0.020000-0.003000j : [mag:0.020224 ∠-8.530766],nan,Y:0.000020+0.000060j : [mag:0.000063 ∠71.565051]


,Base,Emitter,Collector
Base,Y:0.002000+0.002000j : [mag:0.002828 ∠45.000000],Y:-0.001998-0.001980j : [mag:0.002813 ∠-135.259255],Y:-0.000002-0.000020j : [mag:0.000020 ∠-95.710593]
Emitter,Y:-0.022000+0.001000j : [mag:0.022023 ∠177.397438],Y:0.022018-0.000960j : [mag:0.022039 ∠-2.496554],Y:-0.000018-0.000040j : [mag:0.000044 ∠-114.227745]
Collector,Y:0.020000-0.003000j : [mag:0.020224 ∠-8.530766],Y:-0.020020+0.002940j : [mag:0.020235 ∠171.645649],Y:0.000020+0.000060j : [mag:0.000063 ∠71.565051]
